In [1]:
#upload attention.py and contractions.py
from google.colab import files
files.upload()

Saving attention.py to attention.py
Saving contractions.py to contractions.py


{'attention.py': b'import tensorflow as tf\r\nimport os\r\nfrom tensorflow.python.keras.layers import Layer\r\nfrom tensorflow.python.keras import backend as K\r\n\r\n\r\nclass AttentionLayer(Layer):\r\n    """\r\n    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).\r\n    There are three sets of weights introduced W_a, U_a, and V_a\r\n     """\r\n\r\n    def __init__(self, **kwargs):\r\n        super(AttentionLayer, self).__init__(**kwargs)\r\n\r\n    def build(self, input_shape):\r\n        assert isinstance(input_shape, list)\r\n        # Create a trainable weight variable for this layer.\r\n\r\n        self.W_a = self.add_weight(name=\'W_a\',\r\n                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),\r\n                                   initializer=\'uniform\',\r\n                                   trainable=True)\r\n        self.U_a = self.add_weight(name=\'U_a\',\r\n                                   shape=t

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#importing AttetionLayer from the uploaded attention.py
from attention import AttentionLayer

#importing contraction_mapping from contractions
from contractions import contraction_mapping

# Importing required libraries

In [4]:
import numpy as np
import pandas as pd
import re 
from bs4 import BeautifulSoup

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from nltk.corpus import stopwords

from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model 
from tensorflow.keras.callbacks import EarlyStopping 

import warnings  
warnings.filterwarnings("ignore")

Using TensorFlow backend.


# Reading the data

In [0]:
data = pd.read_csv('/content/drive/My Drive/Data/Reviews.csv')

In [6]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


Drop duplicates and NA values


In [0]:
#dropping duplicates
data.drop_duplicates(subset=['Text'], inplace=True)

#dropping na values
data.dropna(axis=0, inplace=True)

# Text cleaning

In [8]:
#Looking at the text
data['Text'][:10]

0    I have bought several of the Vitality canned d...
1    Product arrived labeled as Jumbo Salted Peanut...
2    This is a confection that has been around a fe...
3    If you are looking for the secret ingredient i...
4    Great taffy at a great price.  There was a wid...
5    I got a wild hair for taffy and ordered this f...
6    This saltwater taffy had great flavors and was...
7    This taffy is so good.  It is very soft and ch...
8    Right now I'm mostly just sprouting this so my...
9    This is a very healthy dog food. Good for thei...
Name: Text, dtype: object

In [10]:
import nltk
nltk.download('stopwords')

stop_words= set(stopwords.words('english'))

def text_cleaner(text):
    #Converting everything to lowerr case
    newString = text.lower()

    #Remove HTML tags
    newString = BeautifulSoup(newString, "lxml").text

    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"', '', newString)

    #Contraction mapping
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])

    #Removing 's 
    newString = re.sub(r"'s\b", "", newString)

    newString = re.sub("[^a-zA-Z]", " ", newString)
    tokens = [w for w in newString.split() if w not in stop_words]

    long_words = []
    for i in tokens:
        if len(i) > 3:
            long_words.append(i)
    
    return (" ".join(long_words).strip())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Summary cleaning

In [11]:
#Looking at the summaries
data['Summary'][:10]

0                            Good Quality Dog Food
1                                Not as Advertised
2                            "Delight" says it all
3                                   Cough Medicine
4                                      Great taffy
5                                       Nice Taffy
6    Great!  Just as good as the expensive brands!
7                           Wonderful, tasty taffy
8                                       Yay Barley
9                                 Healthy Dog Food
Name: Summary, dtype: object

In [0]:
def summary_cleaner(text):
    newString = re.sub('"', '', text)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])
    newString = re.sub(r"'s\b", "", newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    newStrig = newString.lower()

    tokens = newString.split()
    newString = ''
    for token in tokens:
        if len(token) > 3:
            newString = newString + token + ' '
    
    return newString

In [0]:
cleaned_text = []
for t in data['Text']:
    cleaned_text.append(text_cleaner(t))

cleaned_summary = []
for t in data['Summary']:
    cleaned_summary.append(summary_cleaner(t))

In [0]:
data['cleaned_text'] = cleaned_text
data['cleaned_summary'] = cleaned_summary

data['cleaned_summary'].replace('', np.nan, inplace=True)
data.dropna(axis=0, inplace=True)

In [21]:
cleaned_data = data[['cleaned_text', 'cleaned_summary']]
cleaned_data.head()

,cleaned_text,cleaned_summary
0,bought several vitality canned food products f...,Good Quality Food
1,product arrived labeled jumbo salted peanuts p...,Advertised
2,confection around centuries light pillowy citr...,Delight says
3,looking secret ingredient robitussin believe f...,Cough Medicine
4,great taffy great price wide assortment yummy ...,Great taffy


In [0]:
#converting cleaned_data dataFrame into a csv file
cleaned_data.to_csv('cleanedData.csv', index = False)

In [0]:
#copying the csv file to google drive
!cp /content/cleanedData.csv /content/drive/My\ Drive/Data